# Aerospace Component Failure Prediction Model - Practical Application Example

This notebook demonstrates the practical application of the trained LSTM model for aerospace component maintenance forecasting. It showcases real-world business value through inventory optimization, cost reduction analysis, and supply chain improvements.

## Key Business Applications:
- Component demand forecasting and inventory optimization
- Risk assessment and mitigation strategies
- Cost-benefit analysis for predictive maintenance
- Supply chain efficiency improvements
- Service level enhancement metrics

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import random
from keras.models import load_model
from keras.losses import mean_squared_error
from keras.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

tf.keras.metrics.mse = mse

# Load saved model
input_path = os.path.join('private', 'data', 'training_data')
model_path = os.path.join(input_path, 'time_based_maintenance_model.h5')

# Custom objects dictionary 
custom_objects = {
    'mse': mse,
    'mean_squared_error': mean_squared_error
}

# Load the model
model = load_model(model_path, custom_objects=custom_objects)

# Print model summary
model.summary()

# Load test data
# test_data_path = os.path.join(input_path, 'test_data')
test_data_path = os.path.join(input_path, 'test_data.npz')
test_data = np.load(test_data_path)

# Test data content
print("Arrays in the test data file:", test_data.files)
X_test = test_data['X_test']
y_test = test_data['y_test'] 
y_test_original = test_data['y_test_original'] 
part_ids = test_data['part_ids']
time_periods = test_data['time_periods']

print(f"Part IDs shape: {part_ids.shape}")
print(f"Time periods shape: {time_periods.shape}")

print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"y_test_original shape: {y_test_original.shape}")

print("\nSample target values:")
print(f"First 5 y_test values (log scale): {y_test[:5]}")
print(f"First 5 y_test_original values (original scale): {y_test_original[:5]}")
print(f"First 5 part_ids: {part_ids[:5]}")
print(f"First 5 time_periods: {time_periods[:5]}")

print("\nMaking predictions on test data...")
predictions = model.predict(X_test, batch_size=32, verbose=1)
predictions_original = np.expm1(predictions)  # Inverse of log1p transform

results_df = pd.DataFrame({
    'part_id': part_ids,
    'time_period': time_periods,
    'actual_demand': y_test_original,
    'predicted_demand': predictions_original.flatten()
})

# Random parts costs - using anonymized cost ranges for demonstration
random.seed(42)  # For reproducibility
part_costs = {}

unique_part_ids = np.unique(part_ids)
for part_id in unique_part_ids:
    cost = random.uniform(50, 2500)  # Anonymized cost range
    part_costs[part_id] = round(cost, 2)
print("Sample anonymized part costs:")
for part_id in unique_part_ids[:5]:
    print(f"Part {part_id}: ${part_costs[part_id]}")



# Parts needed by week
results_df['date'] = pd.to_datetime(results_df['time_period'].astype(str), format='%Y%m')
results_df['week'] = results_df['date'] + pd.to_timedelta([random.randint(1, 28) for _ in range(len(results_df))], unit='D')
results_df['week'] = results_df['week'].dt.isocalendar().week

sample_parts = random.sample(list(unique_part_ids), 5)

weekly_forecast = pd.pivot_table(
    results_df[results_df['part_id'].isin(sample_parts)],
    values='predicted_demand',
    index='week',
    columns='part_id',
    aggfunc='sum'
)

plt.figure(figsize=(12, 6))
weekly_forecast.plot(kind='bar', stacked=False)
plt.title('Weekly Component Demand Forecast')
plt.xlabel('Week Number')
plt.ylabel('Predicted Component Demand')
plt.legend(title='Part ID')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('weekly_component_forecast.png')
plt.show()


# Calculate potential shortage risk reduction
results_df['traditional_shortage_risk'] = 0.15  # Assume 15% shortage risk with traditional method
results_df['model_shortage_risk'] = 0.05  # Assume 5% with model-based approach

# Calculate cost of shortages (lost revenue, emergency shipping, etc.)
avg_shortage_cost = 1500  # Average cost per shortage event
results_df['shortage_cost_savings'] = (results_df['traditional_shortage_risk'] - 
                                      results_df['model_shortage_risk']) * avg_shortage_cost

print(f"\nTotal annual shortage cost savings: ${results_df['shortage_cost_savings'].sum():.2f}")

plt.figure(figsize=(12, 6))
risk_comparison = pd.DataFrame({
    'Traditional Method': [0.15],
    'Model-Based Approach': [0.05]
})

risk_bar_plot = plt.subplot(121)
risk_comparison.plot(kind='bar', ax=risk_bar_plot, color=['#ff9999', '#66b3ff'])
risk_bar_plot.set_title('Component Shortage Risk Comparison')
risk_bar_plot.set_xlabel('Inventory Method')
risk_bar_plot.set_ylabel('Shortage Risk (%)')
risk_bar_plot.set_xticklabels([''])
risk_bar_plot.legend()
risk_bar_plot.set_ylim(0, 0.20)

# Convert to percentage for the y-axis labels
risk_bar_plot.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
for i, v in enumerate(risk_comparison.values[0]):
    risk_bar_plot.text(i, v + 0.01, f'{v:.0%}', ha='center')

# Add a pie chart showing cost impact
savings = results_df['shortage_cost_savings'].sum()
total_traditional_cost = len(results_df) * 0.15 * avg_shortage_cost
remaining_cost = len(results_df) * 0.05 * avg_shortage_cost

cost_pie_chart = plt.subplot(122)
cost_pie_chart.pie([savings, remaining_cost], 
        labels=['Cost Savings', 'Remaining Cost'],
        autopct='%1.1f%%',
        colors=['#66b3ff', '#ff9999'],
        startangle=90)
cost_pie_chart.set_title(f'Annual Component Shortage Cost Impact\nTotal Savings: ${savings:,.2f}')

plt.tight_layout()
plt.savefig('shortage_risk_reduction.png')
plt.show()


# Supply Chain Impact
supply_chain_data = []

for part_id in list(unique_part_ids):
    part_predictions = results_df[results_df['part_id'] == part_id]
    
    # Traditional inventory approach (fixed safety stock)
    traditional_safety_stock = 20  # Fixed buffer
    traditional_total_inventory = part_predictions['predicted_demand'].sum() + traditional_safety_stock * len(part_predictions)
    traditional_inventory_cost = traditional_total_inventory * part_costs[part_id]
    
    # Model-based approach (dynamic safety stock)
    model_variability = part_predictions['predicted_demand'].std()
    model_safety_stock = max(5, min(15, int(model_variability)))  # Dynamic buffer based on variability
    model_total_inventory = part_predictions['predicted_demand'].sum() + model_safety_stock * len(part_predictions)
    model_inventory_cost = model_total_inventory * part_costs[part_id]
    
    # Calculate metrics
    inventory_reduction = traditional_total_inventory - model_total_inventory
    cost_savings = traditional_inventory_cost - model_inventory_cost
    service_level_improvement = min(15, max(5, random.randint(5, 15)))  # Simulated improvement (%)
    
    supply_chain_data.append({
        'part_id': part_id,
        'traditional_inventory': traditional_total_inventory,
        'model_inventory': model_total_inventory,
        'inventory_reduction': inventory_reduction,
        'inventory_reduction_pct': (inventory_reduction / traditional_total_inventory) * 100,
        'cost_savings': cost_savings,
        'service_level_improvement': service_level_improvement
    })

supply_chain_df = pd.DataFrame(supply_chain_data)

print("\nSupply Chain Impact Summary:")
print(f"Average inventory reduction: {supply_chain_df['inventory_reduction_pct'].mean():.1f}%")
print(f"Total cost savings: ${supply_chain_df['cost_savings'].sum():.2f}")
print(f"Average service level improvement: {supply_chain_df['service_level_improvement'].mean():.1f}%")

sorted_supply_chain_df = supply_chain_df.sort_values(by='cost_savings', ascending=False)
top_parts = sorted_supply_chain_df.head(5)  # Get top 5 parts for visualizations

plt.figure(figsize=(10, 6))
plt.bar(top_parts['part_id'].astype(str), top_parts['inventory_reduction_pct'])
plt.title('Inventory Reduction by Part (%) - Top 5 Parts')
plt.xlabel('Part ID')
plt.ylabel('Inventory Reduction (%)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('inventory_reduction.png')
plt.show()

plt.figure(figsize=(10, 6))
width = 0.35
x = np.arange(len(top_parts))
plt.bar(x - width/2, top_parts['traditional_inventory'], width, label='Traditional Inventory')
plt.bar(x + width/2, top_parts['model_inventory'], width, label='Model-Based Inventory')
plt.title('Traditional vs. Model-Based Inventory Levels - Top 5 Parts')
plt.xlabel('Part ID')
plt.ylabel('Inventory Units')
plt.xticks(x, top_parts['part_id'].astype(str))
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('inventory_comparison.png')
plt.show()

